# Statistical comparison of errors

In [ ]:
import os
import warnings

import geopandas as gpd
import matplotlib.pyplot as plt
import momepy as mm
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns

from core import stats, utils

Create a folder for evaluation data.

In [2]:
os.makedirs("../../evaluation/", exist_ok=True)

In [2]:
# which variables to evaluate
eval_vars = [
    "edge_count",
    "edge_length",
    "node_count",
    "avg_degree",
    "stroke_count",
    "stroke_length_sum",
    "stroke_length_max",
]

# which methods to evaluate
methods_to_evaluate = [
    "revised_manual",
    "cityseer",
    "original",
    "osmnx",
    "parenx-voronoi",
    "parenx-skeletonize",
    "neatnet",
]

methods_to_compare = [
    "cityseer",
    "osmnx",
    "parenx-voronoi",
    "parenx-skeletonize",
    "neatnet",
]

Generate evaluation grids and fill the with evaluation variables. At the same time, compute Euclidean distance between each method and a manual baseline.

In [ ]:
for fua in utils.fua_city:
    print(f"Generating for {fua}")
    # read in base data
    meta = utils.read_sample_data()
    geom = meta.loc[meta.eFUA_ID == fua, "geometry"]
    city = meta.loc[meta.eFUA_ID == fua, "eFUA_name"].values[0]

    gdf_orig = utils.read_original(fua)
    proj_crs = gdf_orig.crs

    # Make grid
    base_grid = utils.make_grid(fua, 9, proj_crs)

    # get info on cells with revised data
    deltas = gpd.read_file(f"../../revision/{fua}/deltas_updated.gpkg")

    # read results from all methods into dict
    methods = {}

    for method in methods_to_evaluate:
        print(f"   Reading in results for {method}")
        gdf = utils.read_results(fua, method, proj_crs)

        # print("     getting graph")
        gdf = gdf[~gdf.normalize().duplicated()].copy().reset_index(drop=True)
        G = mm.gdf_to_nx(gdf, length="length", integer_labels=True)

        nodes, edges = mm.nx_to_gdf(G)

        # add node degrees
        # print("     adding node degree")
        nodes = stats.add_node_degree(nodes, G)

        # add stroke IDs
        # print("     measuring coins")
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=UserWarning)
            if method == "parenx-skeletonize":
                coins = mm.COINS(
                    edges.set_geometry(edges.simplify(0.5)),
                    angle_threshold=120,
                    flow_mode=True,
                )
            else:
                coins = mm.COINS(edges, angle_threshold=120, flow_mode=True)
        edges["stroke_id"] = coins.stroke_attribute()
        stroke_gdf = coins.stroke_gdf()

        methods[method] = {}
        methods[method]["gdf"] = gdf
        methods[method]["graph"] = G
        methods[method]["nodes"] = nodes
        methods[method]["edges"] = edges

        ### grid with stats eval for this method only
        grid = base_grid.copy()

        # print("     measuring grid edge")
        # add ratio columns to grid
        grid[["edge_count", "edge_length"]] = grid.apply(
            lambda x: stats.get_edge_stats(edges, x.geometry),  # noqa: B023
            axis=1,
            result_type="expand",
        )

        # print("     measuring grid node")
        grid[["node_count", "node_degrees", "avg_degree"]] = grid.apply(
            lambda x: stats.get_node_stats(nodes, x.geometry),  # noqa: B023
            axis=1,
            result_type="expand",
        )

        # print("     measuring stroke")
        grid[["stroke_count", "stroke_length_sum", "stroke_length_max"]] = grid.apply(
            lambda x: stats.get_stroke_stats(edges, stroke_gdf, x.geometry),  # noqa: B023
            axis=1,
            result_type="expand",
        )

        # add info on which cells have to be verified still
        grid["to_verify_total"] = deltas["to_verify_total"]

        # save grid to dict
        methods[method]["grid"] = grid

        # save to a file
        grid.to_file(f"../../evaluation/{fua}.gpkg", layer=method)

    # get euclidean distance between the distributions
    deltas = {}
    for eval_var in eval_vars:
        deltas[eval_var] = {}
        for method in methods_to_compare:
            delta_comp = (
                methods["revised_manual"]["grid"][eval_var]
                - methods[method]["grid"][eval_var]
            ) ** 2

            delta = np.sqrt((delta_comp).sum())
            deltas[eval_var][method] = delta

    # plot the distance by eval variable
    fig, axs = plt.subplots(7, figsize=(6, 18))
    for i, eval_var in enumerate(eval_vars):
        s = pd.Series(deltas[eval_var])
        s.plot.barh(ax=axs.flat[i])
        axs.flat[i].axvline(s.min(), color="coral", linestyle="--")
        axs.flat[i].set_xlabel(eval_var)
        plt.tight_layout()
        fig.savefig(
            f"../../plots/evaluation/{fua}/euclidean_distance.png",
            dpi=300,
            bbox_inches="tight",
        )
        plt.close()

Generating for 1133
   Reading in results for revised_manual
   Reading in results for cityseer
   Reading in results for original
   Reading in results for osmnx
   Reading in results for parenx-voronoi
   Reading in results for parenx-skeletonize
   Reading in results for neatnet
Generating for 869
   Reading in results for revised_manual
   Reading in results for cityseer
   Reading in results for original
   Reading in results for osmnx
   Reading in results for parenx-voronoi
   Reading in results for parenx-skeletonize
   Reading in results for neatnet
Generating for 4617
   Reading in results for revised_manual
   Reading in results for cityseer
   Reading in results for original
   Reading in results for osmnx
   Reading in results for parenx-voronoi
   Reading in results for parenx-skeletonize
   Reading in results for neatnet
Generating for 809
   Reading in results for revised_manual
   Reading in results for cityseer
   Reading in results for original
   Reading in results 

## Statistical comparison

Compare the results with the manually simplified networks using xi correlation.

In [ ]:
for fua in utils.fua_city:
    data = {}
    for method in methods_to_evaluate:
        data[method] = gpd.read_file(f"../../evaluation/{fua}.gpkg", layer=method)

    # plot the distance by eval variable
    fig, axs = plt.subplots(7, figsize=(6, 18))

    for eval_var in eval_vars:
        stat_result = pd.DataFrame(columns=["statistics", "pvalue"])
        for method in methods_to_compare:
            result = scipy.stats.chatterjeexi(
                data[method][eval_var].fillna(0),
                data["revised_manual"][eval_var].fillna(0),
            )
            stat_result.loc[method] = [result.statistic, result.pvalue]

        if (stat_result.pvalue > 0.01).any():
            print(fua, eval_var, "pvalue issue")
        stat_result.statistics.plot.barh(ax=axs.flat[i])
        axs.flat[i].axvline(stat_result.statistics.max(), color="coral", linestyle="--")
        axs.flat[i].set_xlabel(f"xi for {eval_var}")
        plt.tight_layout()
        fig.savefig(
            f"../../plots/evaluation/{fua}/xi_correlation.png",
            dpi=300,
            bbox_inches="tight",
        )
        plt.close()

Getting tables.

In [36]:
tables = {}
for fua in utils.fua_city:
    tables[fua] = {}
    data = {}
    for method in methods_to_evaluate:
        data[method] = gpd.read_file(f"../../evaluation/{fua}.gpkg", layer=method)

    for i, eval_var in enumerate(eval_vars):
        stat_result = pd.DataFrame(columns=["statistics", "pvalue"])
        for method in methods_to_compare + ["original"]:
            result = scipy.stats.chatterjeexi(
                data[method][eval_var].fillna(0),
                data["revised_manual"][eval_var].fillna(0),
            )
            stat_result.loc[method] = [result.statistic, result.pvalue]

        tables[fua][eval_var] = stat_result

Combinidng the tables.

In [90]:
combined = pd.DataFrame(columns=["variable", "case", "original"] + methods_to_compare)
combined[combined.columns.drop("variable")] = combined[
    combined.columns.drop("variable")
].astype(float)

i = 0
for fua in utils.fua_city:
    for eval_var in eval_vars:
        combined.loc[i] = tables[fua][eval_var].statistics
        combined.loc[i, ["variable", "case"]] = [eval_var, fua]
        i += 1

/tmp/ipykernel_3362710/3036768559.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'edge_count' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  combined.loc[i, ["variable", "case"]] = [eval_var, fua]


Preparing the final output

In [91]:
combined["case"] = combined["case"].map(utils.fua_city)

In [92]:
combined = combined.set_index(["variable", "case"]).sort_index(level=0)
combined

original  cityseer     osmnx  \
variable          case                                           
avg_degree        Aleppo          0.683894  0.818360  0.732887   
                  Auckland        0.439375  0.699569  0.498237   
                  Bucaramanga     0.676782  0.758940  0.697984   
                  Douala          0.581280  0.828115  0.664304   
                  Liège           0.641419  0.780270  0.662997   
                  Salt Lake City  0.391298  0.600047  0.443200   
                  Wuhan           0.636431  0.717158  0.672680   
edge_count        Aleppo          0.635280  0.873189  0.830912   
                  Auckland        0.403283  0.723654  0.584885   
                  Bucaramanga     0.439914  0.839615  0.625581   
                  Douala          0.464844  0.897105  0.803972   
                  Liège           0.575733  0.804240  0.706771   
                  Salt Lake City  0.397923  0.657687  0.626431   
                  Wuhan           0.519426  0.738200  0.698727   
edge_length       Aleppo          0.875160  0.873406  0.873712   
                  Auckland        0.864817  0.865031  0.861626   
                  Bucaramanga     0.895524  0.861450  0.878973   
                  Douala          0.925766  0.938496  0.910692   
                  Liège           0.828329  0.826618  0.819556   
                  Salt Lake City  0.631732  0.640805  0.613787   
                  Wuhan           0.751018  0.724122  0.742451   
node_count        Aleppo          0.481027  0.882868  0.751256   
                  Auckland        0.272266  0.671801  0.451717   
                  Bucaramanga     0.282805  0.831362  0.474431   
                  Douala          0.369398  0.859341  0.691486   
                  Liège           0.464697  0.803883  0.576657   
                  Salt Lake City  0.254606  0.630190  0.529154   
                  Wuhan           0.392385  0.759494  0.609370   
stroke_count      Aleppo          0.878125  0.859868  0.827365   
                  Auckland        0.725617  0.738539  0.651668   
                  Bucaramanga     0.892612  0.827646  0.836100   
                  Douala          0.929293  0.855325  0.781062   
                  Liège           0.818340  0.797375  0.778910   
                  Salt Lake City  0.645879  0.630765  0.550822   
                  Wuhan           0.768715  0.723513  0.694494   
stroke_length_max Aleppo          0.872386  0.843309  0.737198   
                  Auckland        0.819129  0.763319  0.601344   
                  Bucaramanga     0.913375  0.833275  0.676788   
                  Douala          0.930621  0.827105  0.654650   
                  Liège           0.850570  0.814115  0.697604   
                  Salt Lake City  0.778740  0.771506  0.603987   
                  Wuhan           0.927711  0.796216  0.652147   
stroke_length_sum Aleppo          0.841134  0.798713  0.682826   
                  Auckland        0.726223  0.662357  0.542213   
                  Bucaramanga     0.887058  0.796396  0.656150   
                  Douala          0.873784  0.750273  0.587731   
                  Liège           0.783179  0.734519  0.637507   
                  Salt Lake City  0.566652  0.584452  0.414000   
                  Wuhan           0.830773  0.654001  0.531520   

                                  parenx-voronoi  parenx-skeletonize   neatnet  
variable          case                                                          
avg_degree        Aleppo                0.851845            0.774428  0.912391  
                  Auckland              0.745444            0.709857  0.804845  
                  Bucaramanga           0.868573            0.834364  0.911900  
                  Douala                0.898912            0.794438  0.956817  
                  Liège                 0.811241            0.778400  0.850784  
                  Salt Lake City        0.610153            0.444008  0.703854  
                  W

Doing the same for Euclidean distance between distributions

In [117]:
all_deltas = {}
for fua in utils.fua_city:
    all_deltas[fua] = {}
    data = {}
    for method in methods_to_evaluate:
        data[method] = gpd.read_file(f"../../evaluation/{fua}.gpkg", layer=method)

    # get euclidean distance between the distributions
    for eval_var in eval_vars:
        all_deltas[fua][eval_var] = {}
        for method in ["original"] + methods_to_compare:
            delta_comp = (
                data["revised_manual"][eval_var] - data[method][eval_var]
            ) ** 2

            delta = np.sqrt((delta_comp).sum())
            all_deltas[fua][eval_var][method] = delta

In [122]:
combined_deltas = pd.DataFrame(
    columns=["variable", "case", "original"] + methods_to_compare
)
combined_deltas[combined_deltas.columns.drop("variable")] = combined_deltas[
    combined_deltas.columns.drop("variable")
].astype(float)

i = 0
for fua in utils.fua_city:
    for eval_var in eval_vars:
        combined_deltas.loc[i] = [eval_var, fua] + list(
            all_deltas[fua][eval_var].values()
        )
        i += 1

In [123]:
combined_deltas["case"] = combined_deltas["case"].map(utils.fua_city)

In [124]:
combined_deltas = combined_deltas.set_index(["variable", "case"]).sort_index(level=0)

In [125]:
combined_deltas.round(2)

original    cityseer       osmnx  \
variable          case                                                
avg_degree        Aleppo              85.59       31.67       58.65   
                  Auckland            33.70       17.34       25.11   
                  Bucaramanga         65.62       27.13       59.54   
                  Douala              38.65       14.97       26.02   
                  Liège               83.43       35.50       62.22   
                  Salt Lake City      43.51       30.64       35.25   
                  Wuhan              127.35       76.40      104.16   
edge_count        Aleppo            1061.56      222.52      240.78   
                  Auckland          1891.28      128.01      264.50   
                  Bucaramanga       1900.47      130.84      317.79   
                  Douala            1414.40      106.20      243.87   
                  Liège             1718.62      160.82      241.07   
                  Salt Lake City    1252.82      160.90      245.53   
                  Wuhan             1200.25      377.47      397.69   
edge_length       Aleppo           16519.57    15090.91    17315.99   
                  Auckland          6512.24     5125.48     6619.60   
                  Bucaramanga       7616.64     7111.25     8090.81   
                  Douala            5597.17     5225.22     6096.87   
                  Liège            12283.41    11084.78    12342.97   
                  Salt Lake City   16890.06    16414.89    17126.52   
                  Wuhan            23539.23    22053.92    23830.15   
node_count        Aleppo             976.61      143.90      173.04   
                  Auckland          1816.19       71.11      214.34   
                  Bucaramanga       1851.55      107.05      286.16   
                  Douala            1384.78       77.34      222.10   
                  Liège             1651.35       93.15      197.84   
                  Salt Lake City    1171.78       76.76      181.95   
                  Wuhan             1164.40      207.73      233.84   
stroke_count      Aleppo             104.24      118.55      282.17   
                  Auckland            83.91       55.15      145.21   
                  Bucaramanga         59.97       68.91      166.80   
                  Douala              46.78       59.82      165.64   
                  Liège               99.29       81.60      138.88   
                  Salt Lake City      85.60       80.30      175.99   
                  Wuhan              117.53      120.58      227.99   
stroke_length_max Aleppo          274171.86   531032.67   471476.90   
                  Auckland        103500.48   210457.83   148941.63   
                  Bucaramanga     136894.67   237001.63   235645.00   
                  Douala          224463.20   268294.23   300266.03   
                  Liège           232284.04   628972.24   259137.82   
                  Salt Lake City  240447.83   245564.65   290182.83   
                  Wuhan           609814.50   973655.01  1047796.28   
stroke_length_sum Aleppo          400970.27   532750.58   514895.18   
                  Auckland        177731.19   429900.30   241472.85   
                  Bucaramanga     155142.15   229189.78   249912.94   
                  Douala          233675.13   290668.54   333883.65   
                  Liège           811448.09  1042451.16   824507.47   
                  Salt Lake City  331715.47   339930.14   369534.77   
                  Wuhan           971342.45  1580927.87  1549469.03   

                                  parenx-voronoi  parenx-skeletonize  \
variable          case                                                 
avg_degree        Aleppo                   32.94               35.89   
                  Auckland                 19.16               19.29   
                  Bucaramanga              20.66               21.91   
                  Douala                   11.98     